In [ ]:
import urllib.request
import json
import os
import ssl
import pandas as pd

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

def transform(data):
    data['Childish diseases'] = data['Childish diseases'].map({'yes': 1, 'no': 0})
    data['Accident or serious trauma'] = data['Accident or serious trauma'].map({'yes': 1, 'no': 0})
    data['Surgical intervention'] = data['Surgical intervention'].map({'yes': 1, 'no': 0})
    data['High fevers in the last year'] = data['High fevers in the last year'].map({
        'more than 3 months ago': 2,
        'less than 3 months ago': 1,
        'no': 0
    })
    #data['Diagnosis'] = data['Diagnosis'].map({'Normal': 0, 'Altered': 1})
    data['Frequency of alcohol consumption'] = data['Frequency of alcohol consumption'].map({
        'once a week': 1,
        'several times a week': 2,
        'hardly ever or never': 0,
        'every day': 3
    })
    data['Season'] = data['Season'].map({
        'spring': 0,
        'fall': 1,
        'winter': 2,
        'summer': 3
    })
    data['Smoking habit'] = data['Smoking habit'].map({
        'occasional': 0,
        'daily': 1,
        'never': 2,
    })
    return data

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
rawdata = {
    "Season": ["spring"],
    "Childish diseases": ["yes"],
    "Accident or serious trauma": ["yes"],
    "Surgical intervention": ["yes"],
    "High fevers in the last year": ["more than 3 months ago"],
    "Frequency of alcohol consumption": ["once a week"],
    "Smoking habit": ["never"],
    "Age": [30],
    "Hours Sitting per Day": [8]
}
dfrawdata = pd.DataFrame(rawdata)
data_processed = transform(dfrawdata)

data =  {
  "Inputs": {
    #"data": [{"Season":0,"Childish diseases":1,"Accident or serious trauma":1,"Surgical intervention":1,"High fevers in the last year":2,"Frequency of alcohol consumption":1,"Smoking habit":2,"Age":30,"Number of hours spent sitting per day":8}]
    "data": data_processed.to_json(orient='records')
  },
  "GlobalParameters": {
    "method": "predict"
  }
}

body = str.encode(json.dumps(data))

url = 'https://workspace-fertilite-jxvfk.southafricanorth.inference.ml.azure.com/score'
# Replace this with the primary/secondary key or AMLToken for the endpoint
api_key = 'wGiKQhKXs5YazqwkcpkBtpoGfX3JMOZE'
if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")

# The azureml-model-deployment header will force the request to go to a specific deployment.
# Remove this header to have the request observe the endpoint traffic rules
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key), 'azureml-model-deployment': 'automl4f78aa81750-1' }

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))


KeyError: 'Childish diseases'